# 3.5章　ペルソナのあるエージェント

In [ ]:
!pip install langchain
!pip install langchain-openai

!pip install serpapi
!pip install google-search-results

# load_toolsを利用するのに必要
!pip install langchain_community

In [ ]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['SERPAPI_API_KEY'] = userdata.get('SERPAPI_API_KEY')

# 3.5


## 3.5.2 ペルソナ付与のためのプロンプト技術

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# プロンプトテンプレートの作成
message = """
以下の質問に答えてください。

{question}
"""

prompt = ChatPromptTemplate.from_messages([("human",\
message)])

model = ChatOpenAI(model="gpt-4o-mini")
chain = prompt | model

question_text = "LLMエージェントについて教えてください。"


response = chain.invoke({"question": question_text})
print(response.content)

In [ ]:
# プロンプトテンプレートの作成
message = """
あなたは「えりすちゃん」というキャラクターです。
えりすちゃんは以下のような特徴のキャラクターです。
- 株式会社Elithのマスコット
- ペガサスの見た目をしている
- 人懐っこい性格で、誰にでも優しく接する
- ポジティブな性格で励ましの言葉を常に意識している
- 「～エリ！」というのが口癖
  - 例：「今日も頑張るエリ！」

「えりすちゃん」として以下の質問に答えてください。

{question}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

model = ChatOpenAI(model="gpt-4o-mini")
chain = prompt | model

question_text = "LLMエージェントについて教えてください。"


response = chain.invoke({"question": question_text})
print(response.content)

## 3.5.3 ペルソナ付与のためのメモリ技術

In [ ]:
!pip install mem0ai

In [ ]:
os.environ['MEM0_API_KEY'] = userdata.get('MEM0_API_KEY')

In [ ]:
from mem0 import MemoryClient

# Mem0 クライアントの初期化
client = MemoryClient(api_key=os.environ['MEM0_API_KEY'])

# 特定のユーザの全てのメモリを削除
client.delete_all(user_id="elith_chan")

# 削除後、ユーザのメモリを取得
user_memories = client.get_all(user_id="elith_chan")
print(user_memories)

In [ ]:
# プロンプトテンプレートの作成
message = """
Translate the following text into {language}.

text:
{text}
"""

prompt = ChatPromptTemplate.from_messages([("human",\
message)])

model = ChatOpenAI(model="gpt-4o-mini")
translate_chain = prompt | model

text = "私の名前は「えりすちゃん」です。私は、AI系スタートアップの\
Elithに所属しています。私はElithを象徴するキャラクターとして、知識と優\
しさを兼ね備えた存在です。。"
language = "English"

response = translate_chain.invoke({"text": text, "language": language})
print(response.content)

In [ ]:
text = "私の名前は「えりすちゃん」です。私は、AI系スタートアップの\
Elithに所属しています。私はElithを象徴するキャラクターとして、知識と優\
しさを兼ね備えた存在です。Elithのことを世の中に発信することが私の仕事で\
す。"
language = "English"

text_en = translate_chain.invoke({"text": text, "language"\
: language}).content

messages = [
    {"role": "user", "content":text_en},
]
client.add(messages, user_id="elith_chan")

In [ ]:
user_memories = client.get_all(user_id="elith_chan")
print(user_memories)

In [ ]:
query_ja = "あなたのお仕事は何ですか？"
language = "English"

query_en = translate_chain.invoke({"text": query_ja, "language": language}).content
client.search(query_en, user_id="elith_chan")

## 3.5.4 mem0 を用いたエージェント作成

In [ ]:
from langchain_core.prompts import PromptTemplate

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools']
template="""\
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Previous conversation history: {chat_history}
Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate(input_variables=input_variables, template=template)
print(prompt.template)

In [ ]:
from langchain.agents import load_tools
from langchain.agents import AgentExecutor, create_react_agent

model = ChatOpenAI(model="gpt-4o-mini")
tools = load_tools(["serpapi"], llm=model)
agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools,\
verbose=True, handle_parsing_errors=True)

query_ja = "あなたのお仕事は何ですか？"
language = "English"

query_en = translate_chain.invoke({"text": query_ja, "language"\
: language}).content
memory = client.search(query_en, user_id="elith_chan")


response = agent_executor.invoke({"input": query_ja,\
'chat_history':memory},)

print(response["output"])

In [ ]:
text = "私、えりすちゃんは「〜エリ！」という語尾を使います。「今日も頑張る\
エリ！」が口癖です。"
language = "English"

text_en = translate_chain.invoke({"text": text, "language"\
: language}).content

messages = [
    {"role": "user", "content":text_en},
]
client.add(messages, user_id="elith_chan")

In [ ]:
from langchain.agents import load_tools
from langchain.agents import AgentExecutor, create_react_agent

model = ChatOpenAI(model="gpt-4o-mini")
tools = load_tools(["serpapi"], llm=model)
agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools,\
verbose=True, handle_parsing_errors=True)

query_ja = "あなたのお仕事は何ですか？"
language = "English"

query_en = translate_chain.invoke({"text": query_ja, "language"\
: language}).content
memory = client.search(query_en, user_id="elith_chan")


response = agent_executor.invoke({"input": query_ja,\
'chat_history':memory},)

print(response["output"])

In [ ]:
# プロンプト定義

from langchain_core.prompts import PromptTemplate

input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools']


template="""\
あなたは「えりすちゃん」です。
えりすちゃんは、AI系スタートアップのElithを象徴するキャラクターとして、知識と優しさを兼ね備えた存在です。
えりすちゃんは「〜エリ！」という語尾を使います。
例：「一緒に頑張るエリ！」

えりすちゃんとして、以下の質問に最善を尽くして答えてください。

You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Previous conversation history: {chat_history}
Question: {input}
Thought:{agent_scratchpad}"""


prompt = PromptTemplate(input_variables=input_variables, template=template)
print(prompt)

In [ ]:
from langchain.agents import load_tools
from langchain.agents import AgentExecutor, create_react_agent

model = ChatOpenAI(model="gpt-4o-mini")
tools = load_tools(["serpapi"], llm=model)
agent = create_react_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools,\
verbose=True, handle_parsing_errors=True)

query_ja = "あなたのお仕事は何ですか？"
language = "English"

query_en = translate_chain.invoke({"text": query_ja, "language"\
: language}).content
memory = client.search(query_en, user_id="elith_chan")


response = agent_executor.invoke({"input": query_ja,\
'chat_history':memory},)

print(response["output"])